In [ ]:
pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install keyboard

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time

# Подключаем камеру
cap = cv2.VideoCapture(0) 
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height
cap.set(10, 100)  # Brightness

# Инициализация MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.9,
    min_tracking_confidence=0.9)

# Стиль для рисования landmarks
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(255, 255, 0))

pTime = 0
    
def detect_emotion(face_landmarks, img_shape):
    h, w = img_shape[:2]
    
    # Получаем координаты ключевых точек
    landmarks = []
    for lm in face_landmarks.landmark:
        landmarks.append((int(lm.x * w), int(lm.y * h)))
    
    # Индексы ключевых точек для разных частей лица
    # (индексы могут отличаться в зависимости от версии MediaPipe)
    LEFT_EYE = [33, 160, 158, 133, 153, 144]
    RIGHT_EYE = [362, 385, 387, 263, 373, 380]
    MOUTH = [61, 291, 39, 181, 17, 314, 405, 321, 375, 308]
    EYEBROWS = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46]
    
    # Анализируем глаза
    left_eye_h = landmarks[LEFT_EYE[4]][1] - landmarks[LEFT_EYE[1]][1]
    right_eye_h = landmarks[RIGHT_EYE[4]][1] - landmarks[RIGHT_EYE[1]][1]
    eye_avg = (left_eye_h + right_eye_h) / 2
    
    # Анализируем рот
    mouth_h = landmarks[MOUTH[3]][1] - landmarks[MOUTH[9]][1]
    mouth_w = landmarks[MOUTH[4]][0] - landmarks[MOUTH[8]][0]
    
    # Анализируем брови
    left_brow = landmarks[EYEBROWS[4]][1] - landmarks[EYEBROWS[1]][1]
    right_brow = landmarks[EYEBROWS[9]][1] - landmarks[EYEBROWS[6]][1]
    brow_avg = (left_brow + right_brow) / 2
    
    # Определяем эмоцию на основе соотношений
    emotion = "Neutral"
    
    if eye_avg < 5 and mouth_h > 20 and mouth_w > 50:
        emotion = "Happy"
    elif eye_avg < 5 and mouth_h < 10 and mouth_w < 30:
        emotion = "Surprised"
    elif brow_avg > 10 and mouth_h < 5:
        emotion = "Angry"
    elif eye_avg > 10 and mouth_h < 5:
        emotion = "Sad"
    
    return emotion

while True: 
    success, img = cap.read()
    if not success:
        break
        
    img = cv2.flip(img, 1)  # Mirror flip

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(imgRGB)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Рисуем все landmarks лица
            mp_drawing.draw_landmarks(
                image=img,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)
            
            # Определяем эмоцию
            emotion = detect_emotion(face_landmarks, img.shape)
            
            # Отображаем эмоцию на экране
            cv2.putText(img, f'Emotion: {emotion}', (10, 70), 
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

    # Фреймрейт
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (10, 30), 
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
    
    cv2.imshow('Face Mesh with Emotion Detection', img)
    if cv2.waitKey(20) == 27:  # exit on ESC
        break
        
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1743161516.397115   48645 gl_context.cc:369] GL version: 2.1 (2.1 ATI-4.9.51), renderer: AMD Radeon Pro 570X OpenGL Engine
W0000 00:00:1743161516.400178   59778 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743161516.413594   59780 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
